# Multi-layer perceptron
**Important**: Change the kernel to *PROJECT_NAME local*. You can do this from the *Kernel* menu under *Change kernel*.

In [ ]:
%autosave 0
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
import pickle
import os

In [2]:
model_name = "mlp"
aml_dir = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']
train = pd.read_csv(os.path.join(aml_dir, 'nyc_demand_train.csv'), parse_dates=['timeStamp'])
X = train.drop(['demand', 'timeStamp'], axis=1)

Due to the relatively long training times of neural networks, in this example we use one validation set in place of cross-validation. The validation set will be the final 25% of the training dataset. Training takes about 3 minutes.

In [3]:
len_train = int(len(X) * 0.75)
len_valid = len(X) - len_train
test_fold = [-1]*len_train + [0]*len_valid
ps = PredefinedSplit(test_fold)

In [4]:
regr = MLPRegressor(solver='lbfgs', verbose=True)

We first specify a list of network structures to try. Each tuple in this list specifies the number of hidden layers and the number of units in each. For example, (5,0) specifies a network with one hidden layer containing 5 units while (10,10) creates a network with 2 hidden layers, each with 10 units. We then perform grid search to test the various network structures and regularization parameters. 

In [5]:
hidden_layer_size = [(5,), (10,), (15,), (20,), (25,), (30,), (35,), (40,), (10,10), (20,20), (30,30), (40,40)]

In [6]:
param_grid = {'hidden_layer_sizes': hidden_layer_size,
             'alpha': [0.01, 0.1, 1.0, 10.0]}
regr_cv = GridSearchCV(estimator=regr,
            param_grid=param_grid,
            cv=ps,
            scoring='neg_mean_squared_error',
            verbose=2,
            n_jobs=-1)

In [7]:
regr_pipe = Pipeline([('regr_cv', regr_cv)])
regr_pipe.fit(X, y=train['demand'])

Fitting 1 folds for each of 48 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  2.0min finished


Pipeline(memory=None,
     steps=[('regr_cv', GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1...*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=2))])

In [8]:
with open(os.path.join(aml_dir, model_name + '.pkl'), 'wb') as f:
    pickle.dump(regr_pipe, f)

Cross validation results

In [9]:
cv_results = pd.DataFrame(regr_pipe.named_steps['regr_cv'].cv_results_)
cv_results.sort_values(by='rank_test_score', inplace=True)
cv_results.head()

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_alpha,param_hidden_layer_sizes,params,rank_test_score,split0_test_score,split0_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
46,13.107694,0.000000,-6554.520312,-7621.813617,10,"(30, 30)","{'alpha': 10.0, 'hidden_layer_sizes': (30, 30)}",1,-6554.520312,-7621.813617,0.0,0.0,0.0,0.0
40,7.273600,0.015629,-6661.949055,-7696.342914,10,"(25,)","{'alpha': 10.0, 'hidden_layer_sizes': (25,)}",2,-6661.949055,-7696.342914,0.0,0.0,0.0,0.0
42,8.758588,0.015627,-6710.820795,-7844.005497,10,"(35,)","{'alpha': 10.0, 'hidden_layer_sizes': (35,)}",3,-6710.820795,-7844.005497,0.0,0.0,0.0,0.0
14,6.755545,0.015626,-6908.940873,-8131.942532,0.1,"(15,)","{'alpha': 0.1, 'hidden_layer_sizes': (15,)}",4,-6908.940873,-8131.942532,0.0,0.0,0.0,0.0
4,6.664248,0.015625,-7045.279829,-8149.529520,0.01,"(25,)","{'alpha': 0.01, 'hidden_layer_sizes': (25,)}",5,-7045.279829,-8149.529520,0.0,0.0,0.0,0.0
